# Load the dataset using deeplake


In [1]:
!pip install deeplake

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 400 kB 4.9 MB/s 
     |████████████████████████████████| 132 kB 78.0 MB/s 
     |████████████████████████████████| 79 kB 10.2 MB/s 
     |████████████████████████████████| 6.7 MB 69.5 MB/s 
     |████████████████████████████████| 10.3 MB 65.4 MB/s 
     |████████████████████████████████| 79 kB 10.1 MB/s 
     |████████████████████████████████| 140 kB 81.3 MB/s 
     |████████████████████████████████| 132 kB 88.4 MB/s 
     |████████████████████████████████| 52 kB 1.7 MB/s 
     |████████████████████████████████| 127 kB 57.8 MB/s 
  Created wheel for deeplake: filename=deeplake-3.1.4-py3-none-any.whl size=484616 sha256=f28719cb018a9528f4cd86f053321085d3c64cb62dff63eba27208a2df9d3b3f
  Stored in directory: /root/.cache/pip/wheels/cc/7c/08/5cec7180de8f7da04206eedaa259e93f9f0570999b8fba5835
Successfully built deeplake
  Attempting uninstall: urllib3
   

In [2]:
import deeplake
ds = deeplake.load('hub://activeloop/fer2013-train')
ds_test = deeplake.load('hub://activeloop/fer2013-public-test')

ds.visualize(height=500, width=800)

hub://activeloop/fer2013-train loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/fer2013-train
hub://activeloop/fer2013-public-test loaded successfully.
This dataset can be visualized in Jupyter Notebook by ds.visualize() or at https://app.activeloop.ai/activeloop/fer2013-public-test


 * Serving Flask app "dataset_visualizer" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


# Now build the model

the following structure is just a suggestion and you can do it in your own way.

In [3]:
import tensorflow as tf
from tensorflow import keras
from keras.activations import relu
from keras.layers import *
from keras import Model
from keras import layers as Layers
import numpy as np

To simplify the code we can build the residual block separately:

In [4]:
class ResBlock(tf.keras.Model):
    def __init__(self, channels, stride=1):
        super(ResBlock, self).__init__(name='ResBlock')
        self.conv1 = tf.keras.layers.Conv2D(channels, 3, strides=stride, padding='same')
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(channels, 3, strides=1, padding='same')
        self.bn2 = tf.keras.layers.BatchNormalization()
        if stride != 1:
            self.downsample = tf.keras.Sequential([
                tf.keras.layers.Conv2D(channels, 1, strides=stride),
                tf.keras.layers.BatchNormalization()
            ])
        else:
            self.downsample = lambda x: x

    def call(self, x):
        h = self.conv1(x)
        h = self.bn1(h)
        h = tf.nn.relu(h)
        h = self.conv2(h)
        h = self.bn2(h)
        h += self.downsample(x)
        return tf.nn.relu(h)

Now we build the ResNet Model with ResBlocks

In [5]:
class ResNet34(tf.keras.Model):
    def __init__(self):
        super(ResNet34, self).__init__(name='ResNet34')
        self.conv1 = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same', use_bias=False)
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.pool1 = tf.keras.layers.MaxPool2D(pool_size=3, strides=2, padding='same')

        self.resblock_1 = ResBlock(64)
        self.resblock_2 = ResBlock(64)
        self.resblock_3 = ResBlock(64)
        self.resblock_4 = ResBlock(64)
        self.resblock_5 = ResBlock(64)
        self.resblock_6 = ResBlock(128, stride=2)
        self.resblock_7 = ResBlock(128)
        self.resblock_8 = ResBlock(128)
        self.resblock_9 = ResBlock(128)
        self.resblock_10 = ResBlock(128)
        self.resblock_11 = ResBlock(128)
        self.resblock_12 = ResBlock(256, stride=2)
        self.resblock_13 = ResBlock(256)
        self.resblock_14 = ResBlock(256)
        self.resblock_15 = ResBlock(256)
        self.resblock_16 = ResBlock(256)
        self.resblock_17 = ResBlock(256)
        self.resblock_18 = ResBlock(256)
        self.resblock_19 = ResBlock(256)
        self.resblock_20 = ResBlock(256)
        self.resblock_21 = ResBlock(256)
        self.resblock_22 = ResBlock(512, stride=2)
        self.resblock_23 = ResBlock(512)
        self.resblock_24 = ResBlock(512)
        self.resblock_25 = ResBlock(512)
        self.resblock_26 = ResBlock(512)
        self.resblock_27 = ResBlock(512)
        self.resblock_28 = ResBlock(512)
        self.resblock_29 = ResBlock(512)
        self.resblock_30 = ResBlock(512)
        self.resblock_31 = ResBlock(512)
        self.resblock_32 = ResBlock(512)
        self.resblock_33 = ResBlock(512)
        
        self.avgpool = tf.keras.layers.GlobalAveragePooling2D()
        self.fc = tf.keras.layers.Dense(7, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = tf.nn.relu(x)
        x = self.pool1(x)

        for i in range(1, 34):
            resblock = getattr(self, 'resblock_{}'.format(i))
            x = resblock(x)

        x = self.avgpool(x)
        x = self.fc(x)
        return x

In [6]:
model = ResNet34()
model.build(input_shape=(None, 48, 48, 1))

In [7]:
model.summary()

Model: "ResNet34"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  3136      
                                                                 
 batch_normalization (BatchN  multiple                 256       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 ResBlock (ResBlock)         multiple                  74368     
                                                                 
 ResBlock (ResBlock)         multiple                  74368     
                                                                 
 ResBlock (ResBlock)         multiple                  743

Now compile the model with a proper optimizer and loss function.

In [8]:
model.compile(optimizer='Adam', loss='CategoricalCrossentropy', metrics=['accuracy'])

then fit your model with enough epochs.

In [9]:
train_x = tf.expand_dims(ds.images, axis=-1)
train_x = tf.cast(x=train_x,dtype=tf.float32)
train_y = tf.keras.utils.to_categorical(ds.labels)

In [10]:
model.fit(train_x, train_y, batch_size=32, validation_split=0.15, epochs=1)

763/763 [==============================] - 11033s 14s/step - loss: 1.9187 - accuracy: 0.3207 - val_loss: 6.0386 - val_accuracy: 0.1433
